In [69]:
import pandas as pd
import nltk
import re
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Normalization

# Index
* [Encoding the Target](#Encoding-the-Target)
* [Split](#Split)
* [Train the Model](#Train-the-Model)
* [Testing](#Testing)
* [Saving the Model](#Saving-the-Model)

# Encoding the Target

[Index](#Index)

[`regression`](https://www.tensorflow.org/tutorials/keras/regression)
[`layers/Normalization`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Normalization)

In [70]:
dataframe=pd.read_csv('/tf/ML-project/data/processedDataframe.csv')

In [71]:
# Ejemplo con LabelEncoder
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(dataframe["Category"])
# Ahora `y_encoded` contiene valores numéricos para cada animal

In [72]:
dataframe.drop(columns=["Category"]).head(2)

,Nutrient Data Bank Number,Data.Alpha Carotene,Data.Ash,Data.Beta Carotene,Data.Beta Cryptoxanthin,Data.Carbohydrate,Data.Cholesterol,Data.Choline,Data.Fiber,Data.Kilocalories,...,Data.Major Minerals.Potassium,Data.Major Minerals.Sodium,Data.Major Minerals.Zinc,Data.Vitamins.Vitamin A - IU,Data.Vitamins.Vitamin A - RAE,Data.Vitamins.Vitamin B12,Data.Vitamins.Vitamin B6,Data.Vitamins.Vitamin C,Data.Vitamins.Vitamin E,Data.Vitamins.Vitamin K
0,1001,0,2.11,158,0,0.06,215,19,0.0,717,...,24,576,0.09,2499,684,0.17,0.003,0.0,2.32,7.0
1,1002,0,2.11,158,0,0.06,219,19,0.0,717,...,26,827,0.05,2499,684,0.13,0.003,0.0,2.32,7.0


# Split
[Index](#Index)

In [73]:
X, y = dataframe.drop(columns=["Category"]), y_encoded

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

# Train the Model
[Index](#Index)

In [74]:
# Número de características de entrada y clases de salida
n_features = len(X.columns)
n_classes = pd.Series(y_encoded).nunique()

# Crear el modelo
model = Sequential([
    Normalization(input_shape=(n_features,)),  # Capa de normalización
    Dense(128, activation='relu'),  # Primera capa oculta
    Dense(256, activation='relu'),  # Segunda capa oculta
    Dense(256, activation='relu'),  # Segunda capa oculta
    Dense(128, activation='relu'),  # Tercera capa oculta
    Dense(n_classes, activation='softmax')  # Capa de salida
])

# Antes de compilar el modelo, debes adaptar la capa de Normalization a tus datos
# Esto calculará la media y la desviación estándar de tus datos de entrenamiento
# y los usará para la normalización durante el entrenamiento e inferencia.

# Supongamos que 'X_train' son tus datos de entrenamiento
model.layers[0].adapt(X_train.to_numpy())

# Compilar el modelo
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Resumen del modelo
model.summary()

# Entrenar el modelo como antes
# model.fit(X_train, y_train, epochs=30, batch_size=32, validation_split=0.2)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/preprocessing/normalization.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ normalization_7 (Normalization) │ (None, 41)             │            83 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_26 (Dense)                │ (None, 128)            │         5,376 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_27 (Dense)                │ (None, 256)            │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_28 (Dense)                │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_29 (Dense)                │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_30 (Dense)                │ (None, 669)            │        86,301 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 223,472 (872.94 KB)

 Trainable params: 223,389 (872.61 KB)

 Non-trainable params: 83 (336.00 B)

In [75]:
# Entrenar el modelo
history = model.fit(X_train, y_train,
                    epochs=30,
                    batch_size=32,
                    validation_split=0.2)  # Usa el 20% de los datos para validación


Epoch 1/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - accuracy: 0.1291 - loss: 5.5008 - val_accuracy: 0.2807 - val_loss: 4.2886
Epoch 2/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.2809 - loss: 3.6341 - val_accuracy: 0.3461 - val_loss: 3.7139
Epoch 3/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.3801 - loss: 2.8764 - val_accuracy: 0.4155 - val_loss: 3.3801
Epoch 4/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.4467 - loss: 2.3722 - val_accuracy: 0.4658 - val_loss: 3.3321
Epoch 5/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.5109 - loss: 2.0287 - val_accuracy: 0.4799 - val_loss: 3.2674
Epoch 6/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.5551 - loss: 1.7570 - val_accuracy: 0.5221 - val_loss: 3.2543
Epoch 7/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.6129 - loss: 1.4631 - val_accuracy: 0.5553 - val_loss: 3.3317
Epoch 8/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.6380 - loss: 1.3153 - val_accuracy

# Testing
[Index](#Index)

In [76]:
# Evaluar el modelo con el conjunto de datos de prueba
test_loss, test_accuracy = model.evaluate(X_test, y_test)

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")


77/77 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6213 - loss: 4.9715
Test Loss: 4.8885111808776855
Test Accuracy: 0.6232120990753174


# Saving the Model

In [78]:
model.save('../models/my_model.keras')